# Other cuda features

## Atomic operations

The [official CUDA documentation](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#atomic-functions) refers to atomic operations as:

> "The operation is atomic in the sense that it is guaranteed to be performed without interference from other threads. In other words, no other thread can access this address until the operation is complete"

Atomic operations function as _locks_ and avoid race conditions. This is especially useful for CUDA programms which primarily work in parallel.

Supported operations in Numba are: `add, compare_and_swap, max, min, nanmax, nanmin, sub`

In [26]:
from numba import cuda
import numpy as np

@cuda.jit
def find_max_value(result, input):
    """
        Find the maximum value of the input array
    """
    
    i = cuda.grid(1)
        
    # Is i the new minimum value?
    cuda.atomic.max(result, 0, input[i])
    
# Array of random values
inArray = np.random.rand(16384)
result = np.zeros(1, dtype=np.float64)

find_max_value[256, 64](result, inArray)

print(f"Maximum value in Array using atomic operations:\n{result[0]}")
print(f"Maximum value in Array using simple python:\n{max(inArray)}")

Maximum value in Array using atomic operations:
0.9998963561775785
Maximum value in Array using simple python:
0.9998963561775785


## Select device to use

As show in the intro, Numba provides ways to select the GPU. Nevertheless, most of the times this should not be needed , as Numba automatically chooses the fastest devcice available.

In [24]:
from numba import cuda

cuda.select_device(0) # Default = fastest

# cuda.select_device(1) => Having more than one GPU available

print(f"Avvailable GPUs: {cuda.gpus}")
print(f"GPU currently in use: {cuda.gpus.current}")

Avvailable GPUs: <Managed Device 0>
GPU currently in use: <Managed Device 0>
